In [1]:
import pandas as pd
# from geopy import distance

# ANALYSIS

In [2]:
# importing the cluster's dataframes
outliers = pd.read_csv('../data/outliers.csv', index_col = 0)
big_cluster = pd.read_csv('../data/big_cluster.csv', index_col = 0)

In [3]:
outliers.head()

,Date,Race,Circuit,Country,Latitude,Longitude,DBSCAN_Clusters
0,2020-03-08,Grand Prix of Qatar,Losail International Circuit,QATAR,25.491000,51.452068,-1
1,2020-03-22,OR Thailand Grand Prix,Buriram International Circuit,THAILAND,14.963057,103.085589,-1
2,2020-04-05,Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES,30.138715,-97.636410,-1
3,2020-04-19,Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA,-27.495925,-64.864078,-1
4,2020-10-18,Motul Grand Prix of Japan,Twin Ring Motegi,JAPAN,36.533647,140.229985,-1


In [4]:
big_cluster.head()

,Date,Race,Circuit,Country,Latitude,Longitude,DBSCAN_Clusters,Subcluster
0,2020-05-03,Gran Premio Red Bull de España,Circuito de Jerez,SPAIN,36.694447,-6.156317,0,1
1,2020-05-17,SHARK Helmets Grand Prix de France,Le Mans,FRANCE,48.007350,0.196738,0,3
2,2020-05-31,Gran Premio d'Italia Oakley,Autodromo Internazionale del Mugello,ITALY,43.999382,11.372365,0,0
3,2020-06-07,Gran Premi Monster Energy de Catalunya,Circuit de Barcelona-Catalunya,SPAIN,41.569469,2.258063,0,1
4,2020-06-21,HJC Helmets Motorrad Grand Prix Deutschland,Sachsenring,GERMANY,50.732860,12.475105,0,0


In [5]:
big_cluster.groupby('Subcluster').size()

Subcluster
0    5
1    4
3    3
dtype: int64

#### I will start optimizing internal routes for subclusters.

## Optimizing routes

As the result of applying mlrose is in 'units' and the units of our coordinates are degrees of Latitude and Longitude, it is better to apply mlrose with a matrix of distances previously calculated in km with geopy.

In [6]:
subclusters = big_cluster['Subcluster'].unique()

In [7]:
subclusters

array([1, 3, 0])

In [8]:
from functions import create_dist_list

In [10]:
# creating dist lists for all subclusters, applying the function 'create_dist_list':
subcluster_dist_dict = {}
for cluster in subclusters:
    subcluster = big_cluster.loc[big_cluster['Subcluster']==cluster].reset_index(drop=True)
    subcluster_dist_dict[cluster] = create_dist_list(subcluster)

In [11]:
subcluster_dist_dict

{1: [(0, 1, 906.0416003864943),
  (0, 2, 709.20506184732),
  (0, 3, 576.0433647759187),
  (1, 2, 213.3824218268996),
  (1, 3, 335.7240942715326),
  (2, 3, 179.43584263435042)],
 3: [(0, 1, 710.3446224068731),
  (0, 2, 460.3195563371287),
  (1, 2, 521.1997578348936)],
 0: [(0, 1, 753.2093789744465),
  (0, 2, 697.4292716963196),
  (0, 3, 445.16349820143387),
  (0, 4, 105.31854933139327),
  (1, 2, 331.6271130393199),
  (1, 3, 424.56157676462203),
  (1, 4, 752.9087233093135),
  (2, 3, 254.1048870356972),
  (2, 4, 650.6950903273602),
  (3, 4, 396.9993782118908)]}

In [13]:
# Calculating best routes:
from functions import best_route

routes_dict = {}

for cluster in subclusters:
    best_order, best_dist = best_route(subcluster_dist_dict[cluster])
    routes_dict[cluster] = (best_order, best_dist)

/usr/local/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [14]:
routes_dict

{1: (array([3, 1, 2, 0]), 1834),
 3: (array([2, 1, 0]), 1692),
 0: (array([1, 0, 4, 3, 2]), 1841)}

### At this point, I have the best routes for the circuits inside the 3 subclusters. Now I will calculate the best route for the outliers

In [15]:
outliers_dist_list = create_dist_list(outliers)

In [16]:
outliers_dist_list

[(0, 1, 5486.3451291389265),
 (0, 2, 13027.280851968753),
 (0, 3, 13748.46427144216),
 (0, 4, 8270.183074011635),
 (0, 5, 12038.62218753436),
 (0, 6, 5934.373927069669),
 (0, 7, 4365.819538710465),
 (0, 8, 4365.819538710465),
 (1, 2, 14545.997308130585),
 (1, 3, 18150.991002447245),
 (1, 4, 4384.353204179663),
 (1, 5, 7369.134897106193),
 (1, 6, 1357.7900737827379),
 (1, 7, 7839.468845562157),
 (1, 8, 7839.468845562157),
 (2, 3, 7269.066592961213),
 (2, 4, 10479.282482273198),
 (2, 5, 14274.208157616871),
 (2, 6, 15827.81888198896),
 (2, 7, 8726.736191782999),
 (2, 8, 8726.736191782999),
 (3, 4, 17459.966421628003),
 (3, 5, 12058.974243919298),
 (3, 6, 16923.7296460576),
 (3, 7, 12706.666915492708),
 (3, 8, 12706.666915492708),
 (4, 5, 8323.338197187517),
 (4, 6, 5435.523367568314),
 (4, 7, 7663.212564868035),
 (4, 8, 7663.212564868035),
 (5, 6, 6386.1405068187905),
 (5, 7, 15179.42626628677),
 (5, 8, 15179.42626628677),
 (6, 7, 8943.668223441835),
 (6, 8, 8943.668223441835),
 (7, 8, 0

In [17]:
outliers

,Date,Race,Circuit,Country,Latitude,Longitude,DBSCAN_Clusters
0,2020-03-08,Grand Prix of Qatar,Losail International Circuit,QATAR,25.491000,51.452068,-1
1,2020-03-22,OR Thailand Grand Prix,Buriram International Circuit,THAILAND,14.963057,103.085589,-1
2,2020-04-05,Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES,30.138715,-97.636410,-1
3,2020-04-19,Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA,-27.495925,-64.864078,-1
4,2020-10-18,Motul Grand Prix of Japan,Twin Ring Motegi,JAPAN,36.533647,140.229985,-1
5,2020-10-25,Australian Motorcycle Grand Prix,Phillip Island,AUSTRALIA,-38.496688,145.238917,-1
6,2020-11-01,Shell Malaysia Motorcycle Grand Prix,Sepang International Circuit,MALAYSIA,2.760191,101.736859,-1
7,2020-07-12,Finland Grand Prix,KymiRing,FINLAND,60.881176,26.478826,0
8,2020-07-12,Finland Grand Prix,KymiRing,FINLAND,60.881176,26.478826,0


### Subcluster 2

In [ ]:
# creating a list with distances between circuits:
subcluster2 = circuits.query('Subcluster == 2').reset_index(drop=True)

dist_list2 = []

for i in subcluster2.index:
    for j in subcluster2.index:
        if i < j: 
            coord_i = (subcluster2.loc[i,'Latitude'],subcluster2.loc[i,'Longitude'])
            coord_j = (subcluster2.loc[j,'Latitude'],subcluster2.loc[j,'Longitude'])

            dist = distance.distance(coord_i, coord_j).km

            dist_list2.append((i, j, dist))

In [ ]:
dist_list2

In [ ]:
# defining a function to apply all the steps for mlrose

import math

def route_calc(dist_list):
    # Initialize fitness function object using coords_sub0
    fitness_coords = mlrose.TravellingSales(distances = dist_list)
    
    # Calculating the number of circuits depending on the number of distances in list
    # The num of distances is equal to n * (n-1) / 2, being n the number of circuits.
    # we want to find n having the num of distances, this is a quadratic function:
    # x**2 - x - 2y = 0, being x = n and y = num of distances.
    # Solving the equation with math library:
    
    a = 1
    b = -1
    c = -2*len(dist_list)
    # calculate the discriminant
    d = (b**2) - (4*a*c)
    # find two solutions
    sol1 = (-b-math.sqrt(d))/(2*a)
    sol2 = (-b+math.sqrt(d))/(2*a)
    
    # assigning the positive solution to length, needed for the algorithm
    length = max(sol1, sol2)
    
    # Define optimization problem object
    problem_fit = mlrose.TSPOpt(length = length, fitness_fn = fitness_coords, 
                                maximize = False)
    
    # Solve using genetic algorithm - attempt 1
    best_state1, best_fitness1 = mlrose.genetic_alg(problem_fit, random_state = 2)
    
    # Solve using genetic algorithm - attempt 2
    best_state2, best_fitness2 = mlrose.genetic_alg(problem_fit, mutation_prob = 0.2, 
                                                  max_attempts = 100, random_state = 2)
    
    if best_fitness1 < best_fitness2:
        return best_state1, round(best_fitness1)
    else:
        return best_state2, round(best_fitness2)

In [ ]:
best_order2, round_dist2 = route_calc(dist_list2)

In [ ]:
best_order2

In [ ]:
# creating the route with circuit names:
best_route2 = []
for i in best_order2:
    name = subcluster2.loc[i, 'Circuit']
    best_route2.append(name)

In [ ]:
best_route2

#### This is a round route. When calculating the distance for the whole calendar, I will remove one step. By now, I will append the distance to the list.

In [ ]:
best_route2.append(round(round_dist2))

In [ ]:
best_route2

### Subcluster 1

In [ ]:
# creating a list with distances between circuits:
subcluster1 = circuits.query('Subcluster == 1').reset_index(drop=True)

dist_list1 = []

for i in subcluster1.index:
    for j in subcluster1.index:
        if i < j: 
            coord_i = (subcluster1.loc[i,'Latitude'],subcluster1.loc[i,'Longitude'])
            coord_j = (subcluster1.loc[j,'Latitude'],subcluster1.loc[j,'Longitude'])

            dist = distance.distance(coord_i, coord_j).km

            dist_list1.append((i, j, dist))

In [ ]:
dist_list1

In [ ]:
# applying the function
best_order1, round_dist1 = route_calc(dist_list1)

In [ ]:
best_order1

In [ ]:
# creating the route with circuit names:
best_route1 = []
for i in best_order1:
    name = subcluster1.loc[i, 'Circuit']
    best_route1.append(name)

In [ ]:
best_route1

In [ ]:
# appending the distance
best_route1.append(round(round_dist1))

In [ ]:
best_route1

### Subcluster 3

In [ ]:
# creating a list with distances between circuits:
subcluster3 = circuits.query('Subcluster == 3').reset_index(drop=True)

dist_list3 = []

for i in subcluster3.index:
    for j in subcluster3.index:
        if i < j: 
            coord_i = (subcluster3.loc[i,'Latitude'],subcluster3.loc[i,'Longitude'])
            coord_j = (subcluster3.loc[j,'Latitude'],subcluster3.loc[j,'Longitude'])

            dist = distance.distance(coord_i, coord_j).km

            dist_list3.append((i, j, dist))

In [ ]:
dist_list3

In [ ]:
# applying the function:
best_order3, round_dist3 = route_calc(dist_list3)

In [ ]:
best_order3

In [ ]:
# creating the route with circuit names:
best_route3 = []
for i in best_order3:
    name = subcluster3.loc[i, 'Circuit']
    best_route3.append(name)

In [ ]:
best_route3

In [ ]:
# appending the round distance
best_route3.append(round(round_dist3))

In [ ]:
best_route3

### At this point, I have the best routes for the circuits inside the 3 subclusters. Still pending 1 circuit alone in the 4th subclusters and 7 circuits with no subcluster

### I will consider this 8 circuits as a new subcluster and calculate the best route between them

In [ ]:
# creating the new dataframe
other_circuits = circuits.loc[(circuits['Subcluster'] == 0) | (circuits['Subcluster'] == 'None')].reset_index(drop=True)

In [ ]:
other_circuits

In [ ]:
dist_list_other = []

for i in other_circuits.index:
    for j in other_circuits.index:
        if i < j: 
            coord_i = (other_circuits.loc[i,'Latitude'],other_circuits.loc[i,'Longitude'])
            coord_j = (other_circuits.loc[j,'Latitude'],other_circuits.loc[j,'Longitude'])

            dist = distance.distance(coord_i, coord_j).km

            dist_list_other.append((i, j, dist))

In [ ]:
dist_list_other

In [ ]:
# applying the function:
best_order_other, round_dist_other = route_calc(dist_list_other)

In [ ]:
best_order_other

In [ ]:
# creating the route with circuit names:
best_route_other = []
for i in best_order_other:
    name = other_circuits.loc[i, 'Circuit']
    best_route_other.append(name)

In [ ]:
best_route_other

In [ ]:
# appending the round distance
best_route_other.append(round(round_dist_other))

In [ ]:
best_route_other

### At this point, I have all the distances between points inside each cluster, considering 'the outliers' like a cluster, but these are round distances.

### Now it's time to make them linear and calculate distances from one cluster to another and sum them all up.

### To do this, as I have one circuit in the 'outliers' which initally was considered in the first calculated cluster, I will consider this circuit as the last stage of the first route. And from this circuit, I will calculate the distance to the nearest circuit of the 3 subclusters.

### I will follow this method until I have all the circuits connected.

In [ ]:
# defining functions to reorder the route according to last or first circuit
import numpy as np   

def reordering_last(array, index):
    position = int(np.where(array == index)[0]) + 1
    return np.roll(array, len(array)-position)

def reordering_first(array, index):
    position = int(np.where(array == index)[0])
    return np.roll(array, -position)

In [ ]:
# defining a function to subtract the distance for the last stage of the round route.

def to_subtract(array, dist_list):
    for el in dist_list:
        if el[0] == array[0] and el[1] == array[-1]:
            return el[2]
        elif el[0] == array[-1] and el[1] == array[0]:
            return el[2]

In [ ]:
# looking for the index of the wanted last circuit
last_circuit = other_circuits.query('Subcluster == 0').index[0]

In [ ]:
# applying functions:
first_stage = reordering_last(best_order_other, last_circuit)
subtract0 = to_subtract(first_stage, dist_list_other)

In [ ]:
# updating the route with circuit names:
first_stage_names = []
for i in first_stage:
    name = other_circuits.loc[i, 'Circuit']
    first_stage_names.append(name)

In [ ]:
# appending the distance subtracting the last step
first_stage_names.append(round(round_dist_other - subtract0))

In [ ]:
first_stage_names

### First stage is defined. Now I will look for the closest circuit to the last circuit of first stage. This will be the starting point for the second stage.

In [ ]:
checking_dist = {}

for idx in circuits.query('Subcluster != "None"').index:
    start_lat = circuits.loc[circuits.loc[circuits['Circuit']== first_stage_names[-2]].index[0],'Latitude']
    start_lon = circuits.loc[circuits.loc[circuits['Circuit']== first_stage_names[-2]].index[0],'Longitude']
    start_coord = (start_lat, start_lon)
    
    i_lat = circuits.loc[idx,'Latitude']
    i_lon = circuits.loc[idx,'Longitude']
    i_coord = (i_lat, i_lon)
    
    dist = distance.distance(start_coord, i_coord).km
    if dist > 0:
        checking_dist[dist] = circuits.loc[idx,'Circuit']

In [ ]:
# checking the closest circuit to the last step for the first stage:
next_circuit = checking_dist[min(checking_dist.keys())]

In [ ]:
# looking for the cluster including the circuit
next_cluster = circuits.loc[circuits.loc[circuits['Circuit'] == next_circuit].index[0]\
                            ,'Subcluster']

Let's check 'next cluster' and reorder it if necessary, to get the route non-rounded

In [ ]:
print('Next circuit should be',next_circuit,'from subcluster', next_cluster)

In [ ]:
# checking the best order for subcluster 2
best_route2

In [ ]:
# as 'Sachsenring' is not in the first place, I need to reorder. I will apply the function
# reorder_first to get that circuit in first place.
# I also will apply the subtract function to get the distance to subtract.

next_circuit_idx = subcluster2.loc[subcluster2['Circuit'] == next_circuit].index[0]
second_stage = reordering_first(best_order2, next_circuit_idx)
subtract2 = to_subtract(second_stage, dist_list2)

In [ ]:
# updating the route with circuit names:
second_stage_names = []
for i in second_stage:
    name = subcluster2.loc[i, 'Circuit']
    second_stage_names.append(name)

In [ ]:
second_stage_names

In [ ]:
# appending the distance subtracting the last step
second_stage_names.append(round(round_dist2 - subtract2))

In [ ]:
second_stage_names

### Second stage is now defined. Again, turn to look for the closest circuit to the last circuit of second stage. This will be the starting point for the third stage.

In [ ]:
checking_dist2 = {}

for idx in circuits.loc[(circuits['Subcluster'] == 1) | (circuits['Subcluster'] == 3)].index:
    start_lat = circuits.loc[circuits.loc[circuits['Circuit']== second_stage_names[-2]].index[0],'Latitude']
    start_lon = circuits.loc[circuits.loc[circuits['Circuit']== second_stage_names[-2]].index[0],'Longitude']
    start_coord = (start_lat, start_lon)
    
    i_lat = circuits.loc[idx,'Latitude']
    i_lon = circuits.loc[idx,'Longitude']
    i_coord = (i_lat, i_lon)
    
    dist = distance.distance(start_coord, i_coord).km
    if dist > 0:
        checking_dist2[dist] = circuits.loc[idx,'Circuit']

In [ ]:
# checking the closest circuit to the last step for the second stage:
next_circuit2 = checking_dist2[min(checking_dist2.keys())]

In [ ]:
# looking for the cluster including the circuit
next_cluster2 = circuits.loc[circuits.loc[circuits['Circuit'] == next_circuit2].index[0]\
                            ,'Subcluster']

In [ ]:
print('Next circuit should be',next_circuit2,'from subcluster', next_cluster2)

In [ ]:
# checking the best order for subcluster 1
best_route1

In [ ]:
# again it is necessary to reorder.
next_circuit2_idx = subcluster1.loc[subcluster1['Circuit'] == next_circuit2].index[0]
third_stage = reordering_first(best_order1, next_circuit2_idx)
subtract1 = to_subtract(third_stage, dist_list1)

In [ ]:
# updating the route with circuit names:
third_stage_names = []
for i in third_stage:
    name = subcluster1.loc[i, 'Circuit']
    third_stage_names.append(name)

In [ ]:
third_stage_names

In [ ]:
# appending the distance subtracting the last step
third_stage_names.append(round(round_dist1 - subtract1))

In [ ]:
third_stage_names

### Third stage defined. One more step to get the whole distances.

In [ ]:
checking_dist3 = {}

for idx in circuits.loc[circuits['Subcluster'] == 3].index:
    start_lat = circuits.loc[circuits.loc[circuits['Circuit']== third_stage_names[-2]].index[0],'Latitude']
    start_lon = circuits.loc[circuits.loc[circuits['Circuit']== third_stage_names[-2]].index[0],'Longitude']
    start_coord = (start_lat, start_lon)
    
    i_lat = circuits.loc[idx,'Latitude']
    i_lon = circuits.loc[idx,'Longitude']
    i_coord = (i_lat, i_lon)
    
    dist = distance.distance(start_coord, i_coord).km
    if dist > 0:
        checking_dist3[dist] = circuits.loc[idx,'Circuit']

In [ ]:
checking_dist3

In [ ]:
# checking the closest circuit to the last step for the third stage:
next_circuit3 = checking_dist3[min(checking_dist3.keys())]

In [ ]:
# looking for the cluster including the circuit. Should be cluster 3, just to confirm
next_cluster3 = circuits.loc[circuits.loc[circuits['Circuit'] == next_circuit3].index[0]\
                            ,'Subcluster']

In [ ]:
next_cluster3

In [ ]:
print('Next circuit should be',next_circuit3,'from subcluster', next_cluster3)

In [ ]:
# checking the best order for subcluster 3
best_route3

In [ ]:
# once more, it is necessary to reorder.
next_circuit3_idx = subcluster3.loc[subcluster3['Circuit'] == next_circuit3].index[0]
fourth_stage = reordering_first(best_order3, next_circuit3_idx)
subtract3 = to_subtract(fourth_stage, dist_list3)

In [ ]:
# updating the route with circuit names:
fourth_stage_names = []
for i in fourth_stage:
    name = subcluster3.loc[i, 'Circuit']
    fourth_stage_names.append(name)

In [ ]:
fourth_stage_names

In [ ]:
# appending the distance subtracting the last step
fourth_stage_names.append(round(round_dist3 - subtract3))

In [ ]:
fourth_stage_names